In [44]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import json


In [45]:
# 1. Load the data
csv_file = "final_processed_data.csv"
geojson_file = "Cleaned_Municipality.geojson"

In [46]:
# Load final_processed_data.csv
df = pd.read_csv(csv_file)
df.rename({'Geometry':'geometry'}, inplace=True)

with open("Cleaned_Municipality.geojson") as f:
    data = json.load(f)
    


# Convert the list to a FeatureCollection
geojson_featurecollection = {
    "type": "FeatureCollection",
    "features": data
}

# Save the fixed GeoJSON
with open("Cleaned_Municipalities_fixed.geojson", "w") as f:
    json.dump(geojson_featurecollection, f)


In [47]:
# 2. Convert the CSV to a GeoDataFrame
geometry = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]
gdf_points = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")  # CRS 4326 is for latitude/longitude

gdf_municipalities = gpd.GeoDataFrame.from_features(geojson_featurecollection["features"], crs="EPSG:4326")

gdf_joined = gpd.sjoin(gdf_points, gdf_municipalities, how="left", predicate="within")


In [48]:
df['Municipality'] = gdf_joined['ref:ISTAT']
df.rename({'geometry':'Geometry'}, inplace=True)
df.drop(columns=['geometry'], inplace=True)

df.to_csv("Tourist_Attractions_Final.csv", index=False)


In [50]:
df.to_csv("Tourist_Attractions_Final.csv", index=False)
